In [6]:
import requests

def identify_tree_species(image_path, api_endpoint, api_key):
    with open(image_path, 'rb') as image_file:
        files = {'file': image_file}
        headers = {'Authorization': f'Bearer {api_key}'}  # Adjust if Pest-ID uses a different auth system
        headers= {}
        response = requests.get(api_endpoint, files=files, headers=headers)
        
        if response.status_code == 200:
            print("Response:", type(response))
            return response.json()  # Assuming JSON response with species info
        else:
            print(f"Failed to identify species for {image_path}. Status code: {response.status_code}")
            return None

# List of image paths
image_paths = ["/home/uav/Downloads/tree2.jpg"]

# Loop through each image and identify tree species
api_endpoint = "https://pest-id.las.iastate.edu/weed/upload"  # Replace with actual endpoint
api_key = "YOUR_API_KEY_HERE"  # Replace with your API key

for image_path in image_paths:
    species_info = identify_tree_species(image_path, api_endpoint, api_key)
    if species_info:
        print(f"Species for {image_path}: {species_info}")


Response: <class 'requests.models.Response'>


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
import cv2
import numpy as np

# Define 3D points in the world coordinate system
object_points = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0]], dtype=np.float32)

# Camera parameters
rvec = np.array([0.1, 0.2, 0.3])  # Rotation vector
tvec = np.array([0.0, 0.0, 5.0])        # Translation vector
camera_matrix = np.array([[800, 0, 320],
                          [0, 800, 240],
                          [0,   0,   1]], dtype=np.float32)
dist_coeffs = np.zeros(5)          # Assume no lens distortion

# Project 3D points to 2D
image_points, _ = cv2.projectPoints(object_points, rvec, tvec, camera_matrix, dist_coeffs)

print("Projected 2D points:\n", image_points)


Projected 2D points:
 [[[320.      240.     ]]

 [[475.3294  290.2849 ]]

 [[275.81876 388.31577]]]


In [2]:
import open3d as o3d

def visualize_pointcloud(filename="pointcloud_data.xyz"):
    # Load point cloud from an ASCII file.
    # The XYZ file is assumed to have one point per line: x y z
    pcd = o3d.io.read_point_cloud(filename, format='xyz')
    
    # Check if pointcloud is successfully loaded
    if not pcd.has_points():
        print("No points found in file or file not found.")
        return
    
    # Visualize the point cloud in a separate window.
    # Use the mouse and keyboard controls in the visualization window:
    #   - Left mouse button drag: Rotate
    #   - Right mouse button drag: Move
    #   - Scroll: Zoom in/out
    #   - 'F': Set a point as the pivot of the rotation
    #   - 'R': Reset view
    #   - 'Q' or ESC: Exit window
    o3d.visualization.draw_geometries([pcd])

visualize_pointcloud("/home/robot_ai/CMU/lab/src/toh_detector/temp/pointcloud.txt")


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [29]:
import open3d as o3d
import numpy as np

def highlight_point_in_cloud(pcd, query_point, highlight_color=[1.0, 1.0, 1.0]):
    # Build a KD-tree for fast nearest-neighbor search
    pcd_tree = o3d.geometry.KDTreeFlann(pcd)

    # Query for the nearest neighbor
    [k, idx, _] = pcd_tree.search_knn_vector_3d(query_point, 1)
    print(f"Closest point index: {idx}")
    print(f"Closest point distance: {k}")
    print(f"Closest point coordinates: {dir(pcd)}")
    # initialize the color array with blue color
    pcd.colors = o3d.utility.Vector3dVector(np.array([[0.0, 0.0, 1.0] for _ in range(len(pcd.points))]))
    if k > 0:
        # idx[0] is the index of the closest point
        highlight_location = pcd.points[idx[0]]
    else:
        print("No nearest point found.")

    return pcd, highlight_location

# Example usage:
# Load your point cloud
pcd = o3d.io.read_point_cloud("/home/robot_ai/CMU/lab/src/toh_detector/temp/pointcloud.txt", format='xyz')
point_to_highlight = np.array([0.0, 0.0, 0.0])  # Replace with your coordinates
highlighted_pcd, highlight_location = highlight_point_in_cloud(pcd, point_to_highlight)
sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.3)
sphere.translate(highlight_location)
sphere.paint_uniform_color([1.0, 0.0, 0.0])  # Red sphere

# Visualize
o3d.visualization.draw_geometries([highlighted_pcd, sphere])


Closest point index: IntVector[26344]
Closest point distance: 1
Closest point coordinates: ['HalfEdgeTriangleMesh', 'Image', 'LineSet', 'PointCloud', 'RGBDImage', 'TetraMesh', 'TriangleMesh', 'Type', 'Unspecified', 'VoxelGrid', '__add__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'clear', 'cluster_dbscan', 'colors', 'compute_convex_hull', 'compute_mahalanobis_distance', 'compute_mean_and_covariance', 'compute_nearest_neighbor_distance', 'compute_point_cloud_distance', 'create_from_depth_image', 'create_from_rgbd_image', 'crop', 'dimension', 'estimate_normals', 'get_axis_aligned_bounding_box', 'get_center', 'get_geometry_type', 'get_max_bound', 'get_min_bound', 'get_oriented_bo